In [43]:
# Activeness Detection by criteria (35.7<T<40, BP-HRV-Stress, SPO2-HR)

import pandas as pd
import numpy as np

TEMPERATURE_THRESHOLD_LOW = 35.8 # degC
TEMPERATURE_THRESHOLD_HIGH = 40 # degC
TIME_THRESHOLD = 30 # min

file_dir = "2023-12-(04-14)_sample.csv"
shift_bin = [0,5,11,17,23]
shift_label = [1,2,3,4]

df_raw = pd.read_csv(file_dir) # The df is sorted time-ascending
df = pd.DataFrame({
    'Time'          : pd.to_datetime(df_raw['Time (WIB)']),
    'Temperature'   : df_raw['Body Temp'],
    'Diastole'      : df_raw['BP Dia'],
    'Systole'       : df_raw['BP Sys'],
    'HRV'           : df_raw['HRV'],
    'Stress'        : df_raw['Stress'],
    'HR'            : df_raw['HR'],
    'SpO2'          : df_raw['SpO2'],
})
df['dt'] = df['Time'].diff().shift(-1).dt.total_seconds()/60
df.loc[df['dt']>20, 'dt'] = 10
prev_label = 0
missing_label_tolerance = 0

def classify_row(row):
    global prev_label, missing_label_tolerance
    if (pd.notna(row['Temperature'])):
        if (row['Temperature'] < 35.7 or row['Temperature'] > 40):
            label = 0
        elif (row['Temperature'] >= 35.7 or row['Temperature'] <= 40):
            label = 1
    elif (row[['Systole', 'Diastole', 'HRV', 'Stress', 'SpO2', 'HR']].notna().any()):
        label = 1
    else:
        missing_label_tolerance += 1
        if missing_label_tolerance <= 3:
            label = prev_label
        else:
            label = 0
            missing_label_tolerance = 0

    prev_label = label
    return label

df['Label'] = np.nan
df['Label'] = df.apply(classify_row, axis=1)
duration_nyala_digunakan = df.loc[df['Label'] == 1, 'dt'].sum()
duration_nyala_tak_digunakan = df.loc[df['Label'] == 0, 'dt'].sum()
print(f"Durasi_nyala_digunakan: {duration_nyala_digunakan}")
print(f"Durasi_nyala_tak_digunakan: {duration_nyala_tak_digunakan}")

df

Durasi_nyala_digunakan: 2151.0
Durasi_nyala_tak_digunakan: 2174.0


,Time,Temperature,Diastole,Systole,HRV,Stress,HR,SpO2,dt,Label
0,2023-11-28 17:23:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10.0,0
1,2023-11-28 17:33:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10.0,0
2,2023-12-01 11:21:00,35.7,NaN,NaN,NaN,NaN,67.0,97.0,10.0,1
3,2023-12-01 11:31:00,36.1,NaN,NaN,NaN,NaN,107.0,NaN,10.0,1
4,2023-12-01 11:41:00,36.1,NaN,NaN,NaN,NaN,NaN,99.0,10.0,1
...,...,...,...,...,...,...,...,...,...,...
633,2023-12-14 16:43:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,1
634,2023-12-14 16:45:00,NaN,71.0,121.0,83.0,33.0,NaN,NaN,6.0,1
635,2023-12-14 16:51:00,NaN,NaN,NaN,NaN,NaN,80.0,NaN,2.0,1
636,2023-12-14 16:53:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,1


In [ ]:
# Trash
# def add3(row):
#     return row['Bod']
# try:
#     df_person = pd.DataFrame({
#         'time': pd.to_datetime(df['time']),
#         'T' : df['TEMPERATURE']
#     })
# except KeyError: continue

# df_person = df_person.sort_values(by='time',ascending=False).query('@TEMPERATURE_THRESHOLD_LOW < T < @TEMPERATURE_THRESHOLD_HIGH')
# df_person['Aktif'] = (df_person['time'] - df_person['time'].shift(-1)).dt.total_seconds()/60
# df_person = df_person.query('Aktif <= 30')
# df_person['Tanggal'] = df_person['time'].dt.date
# df_person['Jam'] = df_person['time'].dt.hour
# df_person['Shift'] = pd.cut(df_person['Jam'], bins=shift_bin, labels=shift_label, right=False)
# _shift = df_person.groupby(['Tanggal', 'Shift']).size().reset_index(name='Jumlah')
# shift = _shift.loc[_shift.groupby('Tanggal')['Jumlah'].idxmax()]

# df_recap = round((df_person.groupby('Tanggal')['Aktif'].sum()/60),1).reset_index()
# df_recap['Nama'] = [f'{name}']*len(df_recap)
# df_recap['Alat'] = [f'{device}']*len(df_recap)
# df_recap = pd.merge(df_recap, shift, on='Tanggal').drop(columns='Jumlah')
# df_recap = df_recap[['Tanggal','Nama', 'Alat','Aktif','Shift']].sort_values(by='Tanggal', ascending=False)

# df_total = pd.concat([df_total, df_recap], ignore_index=True).sort_values(by=['Tanggal','Nama'], ascending=False)

# date_min = df_total['Tanggal'].min()
# date_max = df_total['Tanggal'].max()
# df_total.to_excel(f'Keaktifan_Covwatch_TTL_{date_min}_{date_max}.xlsx',index=False)

# df['Label2'] = df[df['Label1'] == 2]
# df.drop('Label1', axis=1)
    # elif (df['Label'].iloc[max(0,i-1)] == 1 and df['dt'] <= 20): # or \(row['Label'].shift(2) == 1 and row['dt'].shift(1) <= 20):
    #     return 1
    # else :
    #     return 0
# def classify_row_2():
# def check_temperature(_df):
#     return 0 if (_df < 35.7 or _df > 40) else 1
# def check_biomarkers(x):
#     return not pd.isna(x)#1 if (not _df.isna().any()) else np.nan
# df['Label1'] = df['Temperature'].apply(check_temperature)
# df['Label2'] = df[['Systole', 'Diastole', 'HRV', 'Stress', 'SpO2', 'HR']].applymap(check_biomarkers).isna().all()


# df_total = pd.DataFrame({'Tanggal':[],'Nama':[], 'Alat':[],'Aktif':[],'Shift':[]})

# dtype_dict = {
    # "Battery"     : 'int32',      
    # "SpO2"        : 'int32',  
    # "BP Dia"      : 'int32',      
    # "BP Sys"      : 'int32',      
    # "HRV"         : 'int32',  
    # "Stress"      : 'int32',      
    # "HR"          : 'int32',  
    # "Body Temp"   : 'float32',      
    # "Event"       : '',  
    # "Code"        : '',  
    # "Time (WIB)"  : '',          
    # "Name"        : '',  
    # "Device Code" : '',          
# }


# for i, row in df.iterrows():
    # print(i, row)
# def classify_row(i, row):
#     if (row['Temperature'] != np.nan):
#         if (row['Temperature'] < 35.7 or row['Temperature'] > 40):
#             return 0
#         elif (row['Temperature'] >= 35.7 or row['Temperature'] <= 40):
#             return 1
#     elif (row[['Systole', 'Diastole', 'HRV', 'Stress', 'SpO2', 'HR']].notna().any()):
#         return 1
#     else:
#         try:
#             if df['dt'].iloc[max(0,i-1)] < 20 :
#                 return df['Label'].iloc[max(0,i-1)]
#             else:
#                 return 0
#         except:
#             return 0
# df['Label'] = ''

In [190]:
import pandas as pd

# Create a DataFrame
data = {'Name': ['Alice', 'Bob', 'Charlie', 'David', 'Emma'],
        'Age': [25, 30, 35, 40, 45],
        'Gender': ['F', 'M', 'M', 'M', 'F']}
df = pd.DataFrame(data)

# Filter rows where Age is greater than 30
filtered_df = df[df['Age'] > 30]
x = filtered_df.index()
print(filtered_df)


      Name  Age Gender
2  Charlie   35      M
3    David   40      M
4     Emma   45      F


In [1]:
import pandas as pd
from datetime import datetime, timedelta
import csv

# Function to classify a row based on specified rules
def classify_row(row):
    if (row['Body Temp'] < 35.7 or row['Body Temp'] > 40):
        return 0
    elif (not pd.isna(row['BP Dia']) and not pd.isna(row['BP Sys']) and not pd.isna(row['HRV']) and not pd.isna(row['Stress'])):
        return 1
    elif (not pd.isna(row['SpO2']) and not pd.isna(row['HR'])):
        return 1
    else:
        return 0

# Function to calculate duration based on time difference
def calculate_duration(prev_timestamp, curr_timestamp):
    time_diff = curr_timestamp - prev_timestamp
    if time_diff >= timedelta(minutes=20):
        return timedelta(minutes=10)
    else:
        return time_diff

"""
# Example data stream (replace this with your actual data stream mechanism)
data_stream = [
    {'Time': '2024-02-06 12:00:00', 'SpO2': 95, 'BP Dia': 80, 'BP Sys': 120, 'HRV': 60, 'Stress': 0.1, 'HR': 70, 'Body Temp': 36.5},
    {'Time': '2024-02-06 12:10:00', 'SpO2': None, 'BP Dia': None, 'BP Sys': None, 'HRV': None, 'Stress': None, 'HR': 75, 'Body Temp': 35.5},
    {'Time': '2024-02-06 12:20:00', 'SpO2': 98, 'BP Dia': None, 'BP Sys': None, 'HRV': None, 'Stress': None, 'HR': 80, 'Body Temp': 40.5},
    # Add more data rows here...
]

# Convert data stream to DataFrame
df = pd.DataFrame(data_stream)

# Convert 'Time' column to datetime
df['Time'] = pd.to_datetime(df['Time'])

# Initialize variables for duration calculation
prev_timestamp = None
prev_label = None
label_duration = {'0': timedelta(0), '1': timedelta(0)}

# Process incoming data stream
for index, row in df.iterrows():
    # Classify row
    label = classify_row(row)
    
    # Update label duration
    if prev_timestamp is not None:
        duration = calculate_duration(prev_timestamp, row['Time'])
        label_duration[str(prev_label)] += duration
    
    # Update previous timestamp and label
    prev_timestamp = row['Time']
    prev_label = label

# Calculate duration of label 2 (remaining duration)
total_duration = timedelta(hours=24)
label_2_duration = total_duration - label_duration['0'] - label_duration['1']

# Print label durations
print("Duration of Label 0:", label_duration['0']) # Nyala tidak dipakai
print("Duration of Label 1:", label_duration['1']) # Nyala dipakai
print("Duration of Label 2:", label_2_duration) # Tidak nyala
"""

file_path = '2023-12-(04-14)_Label.csv'

# Initialize an empty DataFrame to store daily durations
daily_durations_df = pd.DataFrame(columns=['Date', 'Name', 'Label_0', 'Label_1', 'Label_2'])

# Open the CSV file for reading
with open(file_path, mode='r', newline='') as file:
    reader = csv.DictReader(file)
    
    # Initialize variables to track previous row's data
    prev_name = None
    prev_label = None
    prev_timestamp = None
    
    # Initialize variables to store durations for each Name
    durations = {'0': 0, '1': 0, '2': 0}
    
    # Process each row of the CSV file
    for row in reader:
        timestamp = datetime.strptime(row['Time (WIB)'], '%Y-%m-%d %H:%M:%S')
        if row['Name'] != prev_name:
            if prev_name:
                daily_durations_df = daily_durations_df.append({
                    'Date': prev_timestamp.date(),
                    'Name': prev_name,
                    'Label_0': durations['0'],
                    'Label_1': durations['1'],
                    'Label_2': durations['2']
                }, ignore_index=True)
            durations = {'0': 0, '1': 0, '2': 0}

        label = classify_row(row)
        if prev_timestamp:
            duration = (timestamp - prev_timestamp).total_seconds() // 60  # Convert to minutes
            if duration >= 20:
                duration = 10
            durations[str(prev_label)] += duration
        prev_name = row['Name']
        prev_label = label
        prev_timestamp = timestamp
    
    # Add durations for the last Name to the DataFrame
    if prev_name:
        daily_durations_df = daily_durations_df.append({
            'Date': prev_timestamp.date(),
            'Name': prev_name,
            'Label_0': durations['0'],
            'Label_1': durations['1'],
            'Label_2': durations['2']
        }, ignore_index=True)

print(daily_durations_df)


Duration of Label 0: 0 days 00:10:00
Duration of Label 1: 0 days 00:10:00
Duration of Label 2: 0 days 23:40:00


In [35]:
import numpy as np
x = 1 & 2

print(x)

0
